In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras as K
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive/') # montar google drive com

Mounted at /content/drive/


In [3]:
textos = pd.read_csv("/content/drive/MyDrive/NLP Specialization/MODELO/Historia.csv")

In [4]:
textos.head()

,Unnamed: 0,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,0,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5591.0,4703.0,4207.0,2660.0,3208.0,True,1
1,1,During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5537.0,5242.0,3871.0,2976.0,2402.0,True,1
2,2,Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3862.0,3711.0,3634.0,4219.0,2057.0,False,1
3,3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6783.0,5803.0,3692.0,3209.0,4565.0,True,1
4,4,Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",2765.0,2549.0,2818.0,1018.0,1196.0,False,1


In [5]:
textos = textos.iloc[:,1:]
textos

,TextoCrudo,TextoLimpio,Clase1,Clase2,Clase3,Clase4,Clase5,Predicción,CLASE
0,Alexander III of Macedon Greek: Αλέξανδρος Γʹ ...,"['alexand', 'iii', 'macedon', 'greek', 'αλέξαν...",5591.0,4703.0,4207.0,2660.0,3208.0,True,1
1,During his youth Alexander was tutored by Aris...,"['youth', 'alexand', 'tutor', 'aristotl', 'age...",5537.0,5242.0,3871.0,2976.0,2402.0,True,1
2,Alexander endeavoured to reach the ends of the...,"['alexand', 'endeavour', 'reach', 'end', 'worl...",3862.0,3711.0,3634.0,4219.0,2057.0,False,1
3,Alexander's legacy includes the cultural diffu...,"[""alexander'"", 'legaci', 'includ', 'cultur', '...",6783.0,5803.0,3692.0,3209.0,4565.0,True,1
4,Alexander was born in Pella the capital of the...,"['alexand', 'born', 'pella', 'capit', 'kingdom...",2765.0,2549.0,2818.0,1018.0,1196.0,False,1
...,...,...,...,...,...,...,...,...,...
5937,There are nevertheless a number of contemporar...,"['nevertheless', 'number', 'contemporari', 'hu...",3307.0,3742.0,3302.0,2517.0,4303.0,True,5
5938,Evidence suggests big-game hunter-gatherers cr...,"['evid', 'suggest', 'big-gam', 'hunter-gather'...",2328.0,2236.0,2737.0,1945.0,6431.0,True,5
5939,Hunter-gatherers would eventually flourish all...,"['hunter-gather', 'would', 'eventu', 'flourish...",2399.0,2501.0,2584.0,2496.0,4987.0,True,5
5940,The Archaic period in the Americas saw a chang...,"['archaic', 'period', 'america', 'saw', 'chang...",2129.0,2166.0,2009.0,1808.0,4032.0,True,5


In [7]:
X = textos.iloc[:,2:7].values

In [8]:
Y = textos.iloc[:,8].values

In [9]:
Xn = X / 10000

In [10]:
len(Xn)

5942

In [12]:
modelo = Sequential()
modelo.add(Dense(128,input_shape = Xn.shape[1:]))
modelo.add(Dense(6,activation="softmax"))
modelo.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics =["acc"] )


In [13]:
modelo.load_weights('/content/drive/MyDrive/NLP Specialization/MODELO/Historia.h5')

In [17]:
np.argmax(modelo.predict(Xn[2127].reshape(1, -1))),Y[2127]

(2, 2)

In [18]:
# url = "https://www.history.com/topics/cold-war/cuban-missile-crisis"
# url = "https://courses.lumenlearning.com/suny-hccc-worldhistory/chapter/the-seven-kings/"
# url = "https://heritagecalling.com/2019/02/22/a-brief-introduction-to-prehistoric-caves/"
# url = "https://www.ancient.eu/Medieval_Castle/" FALLO
url = "https://www.britannica.com/event/Battle-of-Tours-732"
# url = "https://www.historytoday.com/archive/1683-siege-vienna"
# url = "https://www.sciencedirect.com/topics/social-sciences/early-modern-times"
# url = "https://www.sparknotes.com/lit/3musk/section15/"  LA PEGO Y NO SE PORQUE
# url = "https://marcopologencana.wordpress.com/2019/06/17/the-crazy-years-between-the-wars/"
# url = "https://www.nationalgeographic.com/news/2006/9/lucy-facts-on-early-human-ancestor/"
# url = "https://www.researchgate.net/publication/236765312_Spartacus_Film_and_History_and_Troy_From_Homer's_'Iliad'_to_Hollywood_Epic_review"
# url = "https://www.thespanishblog.com/2011/06/history-of-spain-the-carthaginians/"
# url = "https://www.history.com/news/prehistoric-ages-timeline"




In [19]:
import pickle
def save_obj(obj, name ):
       with open(name + '.pkl', 'wb') as f:
           pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
       with open('/content/drive/MyDrive/NLP Specialization/MODELO/' + name + '.pkl', 'rb') as f:

           return pickle.load(f)

In [20]:
Vocabulario = load_obj("Vocabulario")
frecuencias = load_obj("Frecuencias")
clases_df = load_obj("clases_df")

In [21]:
clases_df

{1, 2, 3, 4, 5}

In [23]:
from requests import get # la vamos a usar para solicitar la página mediante su URL 
from bs4 import BeautifulSoup # biblioteca para web scraping
import pandas as pdf
import string                              
from nltk.corpus import stopwords # para remover palabras que no agregan significado          
from nltk.stem import PorterStemmer # para llevar las palabras a su forma raíz        
from nltk.tokenize import TweetTokenizer # para transformar cada palabra en un token
from IPython.display import clear_output
from collections import defaultdict
import numpy as np
import nltk
nltk.download('stopwords')

def etiquetas(palabras,frecuencias,clases):
    import numpy as np
    x = np.zeros((1, 6))
    for palabra in palabras:
        for t in clases:
            x[0,t] += frecuencias.get((palabra, t),0)
    x[0,0]=1
    return x

def limpiarTextoCrudo(st):
    res = []
    for t in range(500):
        st = st.replace("[" + str(t) + "]", " ")
    for letra in ["a","b","d","e","f","g","h","i","j"]:
        st = st.replace("[" + letra + "]", " ")
    caracteresARemover = ["(",")","[","]",".",",",'"',"“"]
    for signo in caracteresARemover:
        st = st.replace(signo," ")
    st = st.split()
    hizoJoin = False
    for i in range(len(st) - 1):
        if st[i].isdecimal() and i < len(st) -1 and (
                                   st[i+1].lower() == "ad" or 
                                   st[i+1].lower() == "bc" or 
                                   st[i+1].lower() == "ac" or 
                                   st[i+1].lower() == "dc"):
            res.append("".join([st[i],st[i+1]]))
            hizoJoin = True 
        elif st[i].lower() in {"ad","bc","ac","dc"} and hizoJoin:
            hizoJoin = False    
        elif st[i].lower() not in {"ad","bc","ac","dc"} or not hizoJoin:
            res.append(st[i])
            hizoJoin = False        
    if st[-1].lower() not in {"ad","bc","ac","dc"}: 
        res.append(st[-1])
    return " ".join(res)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stop        = stopwords.words('english')
tokenizer   = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
stemmer     = PorterStemmer()
textosTest  = pd.DataFrame(columns = ['Texto'])
pagina    = get(url)
html      = BeautifulSoup(pagina.text,"html.parser")
parrafos  = [ x.text.strip() for x in html.findAll("p") if x.text != "\n" ]
for p in parrafos:
    palabras = p.split()
    if 40 <= len(palabras):
       clear_output( wait = True )
       print("procesando {}".format(url))
       textoLimpio = limpiarTextoCrudo(p) 
       texto_tokens = tokenizer.tokenize(textoLimpio)
       texto_clean  = [ word for word in texto_tokens if word not in stop and word not in string.punctuation ]
       texto_stem   = [ stemmer.stem(word) for word in texto_clean ]
       textosTest   = textosTest.append({'Texto' : texto_stem}, ignore_index = True)

vocabularioTest = set([x for m in textosTest.index for x in textosTest.loc[m].Texto ])                



for i in clases_df:
    textosTest[f"Clase{i}"] = textosTest.Texto.apply(lambda x : etiquetas(x,frecuencias,clases_df)[0,i])
print("dataset terminado")
print(textosTest.describe())

procesando https://www.britannica.com/event/Battle-of-Tours-732
dataset terminado
            Clase1       Clase2       Clase3       Clase4       Clase5
count    10.000000    10.000000    10.000000    10.000000    10.000000
mean   2738.000000  4155.100000  3252.900000  2996.300000  1845.100000
std    1211.150784  1526.571573  1090.659989  1014.745294  1011.968538
min    1099.000000  2032.000000  1892.000000  1425.000000   561.000000
25%    2006.000000  3254.750000  2530.750000  2447.750000  1349.500000
50%    2337.500000  3671.000000  2971.000000  2698.500000  1578.500000
75%    3534.750000  4911.500000  3955.250000  3732.000000  2624.000000
max    4665.000000  6695.000000  5417.000000  4837.000000  3451.000000


In [25]:
textosTest

,Texto,Clase1,Clase2,Clase3,Clase4,Clase5
0,"[battl, tour, also, call, battl, poitier, octo...",1099.0,2032.0,1892.0,1425.0,594.0
1,"[death, visigoth, king, witiza, 710, left, spa...",1966.0,2905.0,2653.0,2155.0,561.0
2,"[instead, return, north, africa, ṭāriq, march,...",2335.0,3506.0,2490.0,2437.0,1632.0
3,"[aquitain, modern, southwestern, franc, repres...",1818.0,3558.0,2772.0,2480.0,1826.0
4,"[ʿabd, al-raḥmān, charl, seem, perceiv, eud, r...",4665.0,5134.0,4187.0,3862.0,3174.0
5,"[although, battl, describ, length, muslim, chr...",4586.0,6695.0,4369.0,4837.0,3451.0
6,"[frankish, victori, sometim, regard, decis, wo...",2126.0,3171.0,2319.0,2620.0,1300.0
7,"[part, charl, deriv, profit, well, glori, vict...",2340.0,3784.0,3170.0,2777.0,1525.0
8,"[consid, histor, import, battl, tour, import, ...",2598.0,4244.0,3260.0,3342.0,1498.0
9,"[ʿuqba, yusuf, launch, anoth, invas, 739, –, 7...",3847.0,6522.0,5417.0,4028.0,2890.0


In [29]:
XTest = textosTest.iloc[:,1:].values

In [30]:
XTest

array([[1099., 2032., 1892., 1425.,  594.],
       [1966., 2905., 2653., 2155.,  561.],
       [2335., 3506., 2490., 2437., 1632.],
       [1818., 3558., 2772., 2480., 1826.],
       [4665., 5134., 4187., 3862., 3174.],
       [4586., 6695., 4369., 4837., 3451.],
       [2126., 3171., 2319., 2620., 1300.],
       [2340., 3784., 3170., 2777., 1525.],
       [2598., 4244., 3260., 3342., 1498.],
       [3847., 6522., 5417., 4028., 2890.]])

In [31]:
XTestn = XTest / 10000

In [32]:
XTestn


array([[0.1099, 0.2032, 0.1892, 0.1425, 0.0594],
       [0.1966, 0.2905, 0.2653, 0.2155, 0.0561],
       [0.2335, 0.3506, 0.249 , 0.2437, 0.1632],
       [0.1818, 0.3558, 0.2772, 0.248 , 0.1826],
       [0.4665, 0.5134, 0.4187, 0.3862, 0.3174],
       [0.4586, 0.6695, 0.4369, 0.4837, 0.3451],
       [0.2126, 0.3171, 0.2319, 0.262 , 0.13  ],
       [0.234 , 0.3784, 0.317 , 0.2777, 0.1525],
       [0.2598, 0.4244, 0.326 , 0.3342, 0.1498],
       [0.3847, 0.6522, 0.5417, 0.4028, 0.289 ]])

In [33]:
modelo.predict(XTestn[0].reshape(1, -1))

array([[1.3068824e-05, 5.9672431e-03, 7.4587911e-01, 2.0471057e-01,
        4.1956391e-02, 1.4736012e-03]], dtype=float32)

In [34]:
np.argmax(modelo.predict(XTestn[0].reshape(1, -1)))

2

In [35]:
len(vocabularioTest)

465

In [36]:
resultados = {}
for xT in range (len(XTestn)):
  print(np.argmax(modelo.predict(XTestn[xT].reshape(1, -1))),end = " ")
  r = np.argmax(modelo.predict(XTestn[xT].reshape(1, -1)))
  resultados[r] = resultados.get(r, 0) + 1
print()  
resultados

2 2 2 2 2 2 2 2 2 2 


{2: 10}